In [ ]:
#Imports & Model Definition
import torch
import numpy as np
import joblib
import os

class ANNModel(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
#Load Model & Scaler
# Load scaler
scaler = joblib.load("model/scaler.pkl")

# Load model
model = ANNModel(input_dim=19)
model.load_state_dict(torch.load("model/Ann_exam_score_model.pkl", map_location=torch.device('cpu')))
model.eval()
print("✅ Model and scaler loaded.")

In [ ]:
#Mapping Dictionaries
lowmed_map = {'Low': 0, 'Medium': 1, 'High': 2}
yes_no_map = {'Yes': 1, 'No': 0}
school_type_map = {'Public': 0, 'Private': 1}
influence_map = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
education_map = {'None': 0, 'Primary': 1, 'High School': 2, 'College': 3, 'Postgraduate': 4}
distance_map = {'Near': 0, 'Moderate': 1, 'Far': 2}
gender_map = {'Female': 1, 'Male': 0}

In [ ]:
#User Input-Collection
print("🎓 Enter student details:\n")

features = [
    ("Age", "float"),
    ("Hours_Studied", "float"),
    ("Parental_Involvement", "cat", "Low/Medium/High"),
    ("Access_to_Resources", "cat", "Low/Medium/High"),
    ("Extracurricular_Activities", "cat", "Yes/No"),
    ("Sleep_Hours", "float"),
    ("Previous_Scores", "float"),
    ("Motivation_Level", "cat", "Low/Medium/High"),
    ("Internet_Access", "cat", "Yes/No"),
    ("Tutoring_Sessions", "float"),
    ("Family_Income", "cat", "Low/Medium/High"),
    ("Teacher_Quality", "cat", "Low/Medium/High"),
    ("School_Type", "cat", "Public/Private"),
    ("Peer_Influence", "cat", "Positive/Negative/Neutral"),
    ("Physical_Activity", "float"),
    ("Learning_Disabilities", "cat", "Yes/No"),
    ("Parental_Education_Level", "cat", "None/Primary/High School/College/Postgraduate"),
    ("Distance_from_Home", "cat", "Near/Moderate/Far"),
    ("Gender", "cat", "Male/Female")
]

user_input = []

for feat_name, feat_type, *options in features:
    while True:
        if feat_type == "float":
            val = input(f"{feat_name}: ")
            try:
                user_input.append(float(val))
                break
            except:
                print("❌ Invalid number. Try again.")
        else:
            opt_str = options[0]
            val = input(f"{feat_name} [{opt_str}]: ")
            if feat_name == "Parental_Education_Level":
                mapping = education_map
            elif feat_name == "Distance_from_Home":
                mapping = distance_map
            elif feat_name == "Gender":
                mapping = gender_map
            elif feat_name in ['Extracurricular_Activities', 'Internet_Access', 'Learning_Disabilities']:
                mapping = yes_no_map
            elif feat_name in ['Parental_Involvement', 'Access_to_Resources', 'Family_Income', 'Teacher_Quality']:
                mapping = lowmed_map
            elif feat_name == 'School_Type':
                mapping = school_type_map
            elif feat_name == 'Peer_Influence':
                mapping = influence_map
            else:
                mapping = {}

            if val in mapping:
                user_input.append(mapping[val])
                break
            else:
                print(f"❌ Invalid input. Choose from: {opt_str}")

In [ ]:
#Predict & Print
input_array = np.array([user_input], dtype=np.float32)
input_scaled = scaler.transform(input_array)
input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

with torch.no_grad():
    pred = model(input_tensor).item()

print(f"\n🎉 Predicted Exam Score: {pred:.2f}")